https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
from glob import glob

In [12]:
os.listdir("./dataset/mnist_png/")

['testing', 'training']

In [13]:
train_dir = './dataset/mnist_png/training'

test_dir = './dataset/mnist_png/testing'

## Hyperparameter Tunning

In [14]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28, 28, 1)
num_classes = 10

## Preprocess

In [15]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.3,
        zoom_range=0.2,
        horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_directory(
        train_dir, # 디렉토리 경로
        target_size=input_shape[:2], # 채널제외(28,28)
        batch_size=batch_size, 
        color_mode='grayscale',
        class_mode="categorical" # categorical 문제가 아니라면 변경
)

validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=input_shape[:2],
        batch_size=batch_size,
        color_mode='grayscale'
)

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


## Build Model

In [17]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [18]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [20]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator), # 몇개의 이미지가 다 돌아가면 하나의 epoch이냐)
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
1875/1875 [==============================] - 87s 46ms/step - loss: 0.8864 - accuracy: 0.6967 - val_loss: 0.4526 - val_accuracy: 0.8664
Epoch 2/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.5559 - accuracy: 0.8125 - val_loss: 0.3416 - val_accuracy: 0.9079
Epoch 3/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.4527 - accuracy: 0.8532 - val_loss: 0.2198 - val_accuracy: 0.9398
Epoch 4/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.3899 - accuracy: 0.8771 - val_loss: 0.1828 - val_accuracy: 0.9573
Epoch 5/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.3509 - accuracy: 0.8900 - val_loss: 0.1310 - val_accuracy: 0.9630
Epoch 6/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.3350 - accuracy: 0.8965 - val_loss: 0.1282 - val_accuracy: 0.9630
Epoch 7/10
1875/1875 [==========================